In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# legal-easy-markos-1
df = pd.read_csv('../data/legal/input/2024_CSN_Report_Count.csv', skiprows=2)
df = df.dropna()
df['Year'] = df['Year'].astype(int)
df['# of Reports'] = df['# of Reports'].str.replace(',', '').astype(int)
df['diff'] = df['# of Reports'].diff()
df['rel_diff'] = df['diff'] / df['# of Reports'].shift(1)
max_rel_diff_year = df.loc[df['rel_diff'].idxmax(), 'Year']
max_rel_diff_year

2002

In [3]:
# legal-easy-markos-2
df = pd.read_csv("../data/legal/input/2024_CSN_Number_of_Reports_by_Type.csv", skiprows=2)
df = df.dropna()
df['Year'] = df['Year'].astype(int)
for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '').astype(int)
df['total'] = df.iloc[:, 1:].sum(axis=1)
df['fraud_pct'] = df['Fraud '] / df['total']
df[df['fraud_pct'] > 0.5]['Year'].values

array([2010, 2011, 2012, 2013, 2014, 2019])

In [4]:
# legal-easy-markos-3
df = pd.read_csv("../data/legal/input/2024_CSN_Number_of_Reports_by_Type.csv", skiprows=2)
df = df.dropna()
df['Year'] = df['Year'].astype(int)
for col in df.columns[1:]:
    df[col] = df[col].str.replace(',', '').astype(int)
df['total'] = df.iloc[:, 1:].sum(axis=1)
df['Other_pct'] = df['Other '] / df['total']
df['Other_pct'].max() > 0.5

False

In [5]:
# legal-easy-markos-4
cat_df = pd.read_csv('../data/legal/input/2024_CSN_Report_Categories.csv', skiprows=2)
cat_df = cat_df.dropna()
cat_df['Percentage'] = cat_df['Percentage'].str.replace('%', '').astype(float) / 100
cat_df['Rank'] = cat_df['Rank'].astype(int)
cat_df = cat_df.sort_values(by='Percentage', ascending=False)
cat_df['Percentage_cumsum'] = cat_df['Percentage'].cumsum()
cat_df[cat_df['Percentage_cumsum'] > 0.5]['Rank'].values[0]

3

In [6]:
# legal-easy-markos-5
cat_df = pd.read_csv('../data/legal/input/2024_CSN_Report_Categories.csv', skiprows=2)
cat_df = cat_df.dropna()
cat_df[' # of Reports '] = cat_df[' # of Reports '].str.replace(',', '').astype(int)
(cat_df[" # of Reports "].max() / cat_df[" # of Reports "].min()).round(2)

1097.47

In [7]:
# legal-hard-markos-1
state_df = pd.read_csv("../data/legal/input/new_england_states.csv")
state_data = {}
for state in state_df['Name']:
    state = state.replace(" ", "")
    state_data[state] = pd.read_csv(f"../data/legal/raw/csn-data-book-2024-csv/CSVs/State MSA Identity Theft data/{state}.csv", skiprows=2).dropna()
overall_df = pd.concat(state_data.values(), ignore_index=True).reset_index(drop=True)
overall_df['# of Reports'] = overall_df['# of Reports'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x).astype(int)
overall_df = overall_df.sort_values(by='# of Reports', ascending=False)
overall_df = overall_df.drop_duplicates(subset=['Metropolitan Area'], keep='first')
list(overall_df.head(5)["Metropolitan Area"].values)

['Boston-Cambridge-Newton, MA-NH Metropolitan Statistical Area',
 'Providence-Warwick, RI-MA Metropolitan Statistical Area',
 'Hartford-West Hartford-East Hartford, CT Metropolitan Statistical Area',
 'Bridgeport-Stamford-Danbury, CT Metropolitan Statistical Area',
 'Worcester, MA Metropolitan Statistical Area']

In [8]:
# legal-hard-markos-2
state_data = {}
directory = "../data/legal/raw/csn-data-book-2024-csv/CSVs/State MSA Identity Theft data"
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    state_data[filename.split('.')[0]] = pd.read_csv(filepath, skiprows=2).dropna()
overall_df = pd.concat(state_data.values(), ignore_index=True).reset_index(drop=True)
overall_df['states'] = overall_df['Metropolitan Area'].apply(lambda x: x.split(',')[1].split()[0] if ',' in x else None)
overall_df["is_cross_state"] = overall_df['states'].apply(lambda x: True if ('-' in x) else False)
overall_df['# of Reports'] = overall_df['# of Reports'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x).astype(int)
overall_df[overall_df['is_cross_state']]['# of Reports'].sum()

593524

In [9]:
# legal-hard-markos-3
state_data = {}
directory = "../data/legal/raw/csn-data-book-2024-csv/CSVs/State MSA Identity Theft data"
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    state_name = filename.split('.')[0]
    state_data[state_name] = pd.read_csv(filepath, skiprows=2).dropna()
    state_data[state_name]['state_name'] = state_name
overall_df = pd.concat(state_data.values(), ignore_index=True).reset_index(drop=True)
overall_df['# of Reports'] = overall_df['# of Reports'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x).astype(int)
overall_df = overall_df.groupby('state_name').filter(lambda x: len(x) > 1)
overall_df['fraction_of_state'] = overall_df['# of Reports'] / overall_df.groupby('state_name')['# of Reports'].transform('sum')
overall_df.sort_values(by='fraction_of_state', ascending=False).head()['state_name'].values[0]

'Delaware'

In [10]:
# legal-hard-markos-4
df = pd.read_csv('../data/legal/input/2024_CSN_Report_Count.csv', skiprows=2)
df = df.dropna()
df['Year'] = df['Year'].astype(int)
df['# of Reports'] = df['# of Reports'].str.replace(',', '').astype(int)
total_2007_reports = df[df['Year'] == 2007]['# of Reports'].values[0]
print(total_2007_reports)

cat_df = pd.read_csv('../data/legal/input/2024_CSN_Report_Categories.csv', skiprows=2)
cat_df = cat_df.dropna()
auto_related_percentage = cat_df[cat_df['Category'] == 'Auto Related']['Percentage'].values[0]
auto_related_percentage = float(auto_related_percentage.strip('%')) / 100
print(auto_related_percentage)
int(np.round(auto_related_percentage * total_2007_reports))

1070447
0.0304


32542

In [12]:
# legal-hard-markos-5
df = pd.read_csv('../data/legal/input/2024_CSN_Report_Count.csv', skiprows=2)
df = df.dropna()
df['Year'] = df['Year'].astype(int)
df['# of Reports'] = df['# of Reports'].str.replace(',', '').astype(int)
total_2007_reports = df[df['Year'] == 2007]['# of Reports'].values[0]
print(total_2007_reports)

cat_df = pd.read_csv('../data/legal/input/2024_CSN_Report_Categories.csv', skiprows=2)
cat_df = cat_df.dropna()
identity_theft_percentage = cat_df[cat_df['Category'] == 'Identity Theft']['Percentage'].values[0]
identity_theft_percentage = float(identity_theft_percentage.strip('%')) / 100
print(identity_theft_percentage)

id_theft_age_df = pd.read_csv('../data/legal/input/2024_CSN_Identity_Theft_Reports_by_Age.csv', skiprows=2)
id_theft_age_df = id_theft_age_df.dropna()
id_theft_age_df['# of Reports'] = id_theft_age_df['# of Reports'].str.replace(',', '').astype(int)
id_theft_age_df['Percentage of reports'] = id_theft_age_df['# of Reports'] / id_theft_age_df['# of Reports'].sum()
id_theft_over_40 = id_theft_age_df[id_theft_age_df['Age Range'] > '40']['Percentage of reports'].sum()
print(id_theft_over_40)

int(np.round(total_2007_reports * identity_theft_percentage * id_theft_over_40, -3))


1070447
0.1754
0.48607091580930006


91000